# 05. Đánh Giá và Báo Cáo Kết Quả
Notebook này tổng hợp kết quả từ tất cả các mô hình và cung cấp phân tích chi tiết, insight và khuyến nghị.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.utils import shuffle
from src.data.loader import DataLoader
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

## 1. Tải và Chuẩn Bị Dữ Liệu

In [ ]:
# Đặt seed để đảm bảo tính tái lập
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

loader = DataLoader()
df = loader.load_combined_data(merge=False)

# Tạo biến mục tiêu (Pass/Fail)
df['pass_fail'] = df['G3'].apply(lambda x: 1 if x >= 10 else 0)  # 1 là Pass, 0 là Fail

# Chọn các đặc trưng cho mô hình
feature_columns = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 
                  'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 
                  'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 
                  'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

# Mã hóa các biến phân loại
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
label_encoders = {}

for col in feature_columns:
    if df_encoded[col].dtype == 'object':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le

# Tạo tập đặc trưng và nhãn
X = df_encoded[feature_columns]
y = df_encoded['pass_fail']

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Kích thước tập huấn luyện: {X_train_scaled.shape}")
print(f"Kích thước tập kiểm thử: {X_test_scaled.shape}")
print(f"Phân bố lớp trong tập kiểm thử:")
print(pd.Series(y_test).value_counts())

## 2. Huấn Luyện và Đánh Giá Các Mô Hình

In [ ]:
# Định nghĩa các mô hình
models = {
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100),
    'SVM': SVC(random_state=RANDOM_STATE, probability=True)
}

# Huấn luyện và đánh giá các mô hình
results = {}
trained_models = {}
predictions = {}
probabilities = {}

for name, model in models.items():
    print(f"\nĐang huấn luyện mô hình: {name}")
    
    # Huấn luyện mô hình
    model.fit(X_train_scaled, y_train)
    trained_models[name] = model
    
    # Dự đoán
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Lưu dự đoán
    predictions[name] = y_pred
    probabilities[name] = y_pred_proba
    
    # Tính các chỉ số đánh giá
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'y_pred': y_pred,
        'y_true': y_test
    }
    
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-score: {f1:.3f}")
    print(f"ROC-AUC: {roc_auc:.3f}")

## 3. Bảng So Sánh Mô Hình

In [ ]:
# Tạo bảng so sánh
comparison_df = pd.DataFrame(results).T[['accuracy', 'precision', 'recall', 'f1', 'roc_auc']]
comparison_df = comparison_df.round(3)

print("=== BẢNG SO SÁNH CÁC MÔ HÌNH ===")
print(comparison_df)

# Trực quan hóa kết quả
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

for i, (metric, name) in enumerate(zip(metrics, metric_names)):
    axes[i].bar(comparison_df.index, comparison_df[metric], alpha=0.7)
    axes[i].set_title(f'{name} Comparison')
    axes[i].set_ylabel(name)
    axes[i].tick_params(axis='x', rotation=45)

# Hide the unused subplot
axes[5].axis('off')

plt.tight_layout()
plt.show()

## 4. Phân Tích Lỗi Chi Tiết

In [ ]:
# Chọn mô hình tốt nhất theo F1-score
best_model_name = comparison_df['f1'].idxmax()
best_predictions = predictions[best_model_name]
best_probabilities = probabilities[best_model_name]

print(f"Mô hình tốt nhất theo F1-score: {best_model_name}")
print(f"F1-score: {comparison_df.loc[best_model_name, 'f1']:.3f}")

# Phân tích lỗi chi tiết
print(f"\n=== PHÂN TÍCH LỖI CHO MÔ HÌNH {best_model_name.upper()} ===")

# Báo cáo phân loại chi tiết
print(f"\nBáo cáo phân loại:")
print(classification_report(y_test, best_predictions, target_names=['Fail', 'Pass']))

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test, best_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'])
plt.title(f'Ma trận nhầm lẫn - {best_model_name}')
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.show()

# Tính toán các chỉ số từ ma trận nhầm lẫn
tn, fp, fn, tp = cm.ravel()
print(f"\nChi tiết từ ma trận nhầm lẫn:")
print(f"True Negatives (đúng Fail): {tn}")
print(f"False Positives (dự đoán Fail là Pass): {fp}")
print(f"False Negatives (dự đoán Pass là Fail): {fn}")
print(f"True Positives (đúng Pass): {tp}")

# Tỷ lệ lỗi
false_positive_rate = fp / (fp + tn)
false_negative_rate = fn / (fn + tp)
print(f"\nTỷ lệ lỗi dương tính giả: {false_positive_rate:.3f}")
print(f"Tỷ lệ lỗi âm tính giả: {false_negative_rate:.3f}")

## 5. Phân Tích Các Trường Hợp Dự Đoán Sai

In [ ]:
# Tạo dataframe để phân tích lỗi
error_analysis_df = X_test.copy()
error_analysis_df['actual'] = y_test.values
error_analysis_df['predicted'] = best_predictions
error_analysis_df['is_error'] = error_analysis_df['actual'] != error_analysis_df['predicted']
error_analysis_df['probability'] = best_probabilities

# Thêm tên cột gốc để dễ hiểu
original_features = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 
                    'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 
                    'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 
                    'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

print(f"Tổng số trường hợp dự đoán sai: {error_analysis_df['is_error'].sum()}")
print(f"Tỷ lệ dự đoán sai: {error_analysis_df['is_error'].mean():.3f}")

# Phân tích các trường hợp sai
errors = error_analysis_df[error_analysis_df['is_error'] == True]
correct = error_analysis_df[error_analysis_df['is_error'] == False]

print(f"\nChi tiết các loại lỗi:")
fp_cases = errors[(errors['actual'] == 0) & (errors['predicted'] == 1)]  # Dự đoán Pass nhưng thực tế là Fail
fn_cases = errors[(errors['actual'] == 1) & (errors['predicted'] == 0)]  # Dự đoán Fail nhưng thực tế là Pass

print(f"- Sai dương tính (dự đoán Pass, thực tế Fail): {len(fp_cases)} trường hợp")
print(f"- Sai âm tính (dự đoán Fail, thực tế Pass): {len(fn_cases)} trường hợp")

# Trung bình các đặc trưng cho từng loại lỗi
print(f"\nTrung bình các đặc trưng cho trường hợp sai dương tính:")
print(fp_cases[feature_columns].mean())

print(f"\nTrung bình các đặc trưng cho trường hợp sai âm tính:")
print(fn_cases[feature_columns].mean())

## 6. Đường Cong ROC và PR

In [ ]:
# Vẽ đường cong ROC cho tất cả các mô hình
plt.figure(figsize=(15, 5))

# ROC Curves
plt.subplot(1, 2, 1)
for name in models.keys():
    fpr, tpr, _ = roc_curve(y_test, probabilities[name])
    auc = roc_auc_score(y_test, probabilities[name])
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})')
    
plt.plot([0, 1], [0, 1], 'k--', label='Ngẫu nhiên')
plt.xlabel('Tỷ lệ dương tính giả (FPR)')
plt.ylabel('Tỷ lệ dương tính thật (TPR)')
plt.title('Đường cong ROC')
plt.legend()
plt.grid(True)

# PR Curves
plt.subplot(1, 2, 2)
for name in models.keys():
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, probabilities[name])
    avg_precision = average_precision_score(y_test, probabilities[name])
    plt.plot(recall_vals, precision_vals, label=f'{name} (AP = {avg_precision:.3f})')
    
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Đường cong Precision-Recall')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 7. Phân Tích Tính Quan Trọng Của Đặc Trưng (cho Random Forest)

In [ ]:
# Nếu Random Forest là mô hình tốt nhất, phân tích độ quan trọng của đặc trưng
if 'Random Forest' in trained_models:
    rf_model = trained_models['Random Forest']
    feature_importance = rf_model.feature_importances_
    
    # Tạo DataFrame cho độ quan trọng
    importance_df = pd.DataFrame({
        'feature': feature_columns,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(f"=== ĐỘ QUAN TRỌNG CỦA CÁC ĐẶC TRƯNG (Random Forest) ===")
    print(importance_df.head(10))
    
    # Trực quan hóa 10 đặc trưng quan trọng nhất
    plt.figure(figsize=(12, 6))
    top_features = importance_df.head(10)
    plt.barh(range(len(top_features)), top_features['importance'], alpha=0.7)
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Độ quan trọng')
    plt.title('10 Đặc trưng quan trọng nhất theo Random Forest')
    plt.gca().invert_yaxis()  # Hiển thị đặc trưng quan trọng nhất ở trên
    plt.tight_layout()
    plt.show()

## 8. Phân Tích Dữ Liệu Thiếu Nhãn (Semi-supervised)

In [ ]:
# Thử nghiệm mô hình bán giám sát với 5% dữ liệu có nhãn
def create_labeled_unlabeled_data(X, y, labeled_ratio, random_state=None):
    """
    Tạo tập dữ liệu có cả mẫu có nhãn và không có nhãn
    """
    n_total = len(X)
    n_labeled = int(n_total * labeled_ratio)
    
    # Trộn dữ liệu
    X_shuffled, y_shuffled = shuffle(X, y, random_state=random_state)
    
    # Chia thành có nhãn và không nhãn
    X_labeled = X_shuffled[:n_labeled]
    y_labeled = y_shuffled[:n_labeled]
    X_unlabeled = X_shuffled[n_labeled:]
    
    # Tạo nhãn giả cho dữ liệu không nhãn (-1)
    y_unlabeled = np.full(len(X_unlabeled), -1)
    
    # Kết hợp dữ liệu có nhãn và không nhãn
    X_combined = np.vstack([X_labeled, X_unlabeled])
    y_combined = np.hstack([y_labeled, y_unlabeled])
    
    return X_labeled, y_labeled, X_combined, y_combined

# Thử nghiệm với 5% nhãn
X_labeled, y_labeled, X_combined, y_combined = create_labeled_unlabeled_data(
    X_train_scaled, y_train, 0.05, random_state=RANDOM_STATE
)

print(f"\n=== THỬ NGHIỆM BÁN GIÁM SÁT VỚI 5% NHÃN ===")
print(f"Số lượng mẫu có nhãn: {len(y_labeled)}")
print(f"Số lượng mẫu không nhãn: {len(X_combined) - len(y_labeled)}")

# Mô hình chỉ giám sát
supervised_model = LogisticRegression(random_state=RANDOM_STATE)
supervised_model.fit(X_labeled, y_labeled)
y_pred_supervised = supervised_model.predict(X_test_scaled)
supervised_acc = accuracy_score(y_test, y_pred_supervised)
supervised_f1 = f1_score(y_test, y_pred_supervised)

# Mô hình bán giám sát
base_classifier = LogisticRegression(random_state=RANDOM_STATE)
semi_supervised_model = SelfTrainingClassifier(base_classifier, max_iter=10)
semi_supervised_model.fit(X_combined, y_combined)
y_pred_semisupervised = semi_supervised_model.predict(X_test_scaled)
semisupervised_acc = accuracy_score(y_test, y_pred_semisupervised)
semisupervised_f1 = f1_score(y_test, y_pred_semisupervised)

print(f"Supervised-only (5% nhãn): Accuracy={supervised_acc:.3f}, F1={supervised_f1:.3f}")
print(f"Semi-supervised (5% nhãn): Accuracy={semisupervised_acc:.3f}, F1={semisupervised_f1:.3f}")
print(f"Số lượng nhãn giả được sử dụng: {np.sum(semi_supervised_model.transduction_ != -1) - len(y_labeled)}")

## 8.1. Đường cong học bán giám sát (learning curve)

Trong phần này, chúng ta trực quan hóa **F1-score theo tỷ lệ dữ liệu có nhãn** để so sánh:
- Mô hình **chỉ giám sát (supervised-only)**
- Mô hình **Self-Training**
- Mô hình **Label Spreading** (nếu có)

Dữ liệu được lấy từ bảng `semi_supervised_comparison.csv` sinh ra bởi script `scripts/run_semi_supervised.py`. Điều này giúp trả lời câu hỏi: **khi chỉ có 5–50% nhãn thì bán giám sát cải thiện được bao nhiêu so với supervised-only?**

In [ ]:
# Đường cong F1 theo % nhãn: supervised vs self-training vs label spreading

from pathlib import Path

# Đọc bảng kết quả bán giám sát được sinh ra từ scripts/run_semi_supervised.py
project_root = Path('..').resolve()
semi_path = project_root / 'outputs' / 'tables' / 'semi_supervised_comparison.csv'

semi_df = pd.read_csv(semi_path, index_col=0)
semi_df = semi_df.sort_values('labeled_pct')

plt.figure(figsize=(8, 6))
plt.plot(semi_df['labeled_pct'], semi_df['supervised_f1'], marker='o', label='Supervised-only')
plt.plot(semi_df['labeled_pct'], semi_df['self_training_f1'], marker='o', label='Self-Training')

if 'label_spreading_f1' in semi_df.columns:
    plt.plot(semi_df['labeled_pct'], semi_df['label_spreading_f1'], marker='o', label='Label Spreading')

plt.xlabel('Tỷ lệ dữ liệu có nhãn (%)')
plt.ylabel('F1-score')
plt.title('Đường cong học theo % nhãn (Semi-supervised)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Hiển thị bảng tóm tắt F1 theo % nhãn
cols = ['labeled_pct', 'supervised_f1', 'self_training_f1']
if 'label_spreading_f1' in semi_df.columns:
    cols.append('label_spreading_f1')

display(semi_df[cols].round(3))

## 9. 5 Insight Hành Động Cụ Thể

In [ ]:
print("=== 5 INSIGHT HÀNH ĐỘNG CỤ THỂ ===")

# Phân tích các đặc trưng quan trọng từ mô hình tốt nhất
if 'Random Forest' in trained_models:
    rf_model = trained_models['Random Forest']
    feature_importance = rf_model.feature_importances_
    importance_df = pd.DataFrame({
        'feature': feature_columns,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    top_features = importance_df.head(5)['feature'].tolist()
else:
    top_features = ['G2', 'G1', 'failures', 'absences', 'studytime']

# Phân tích các trường hợp dự đoán sai
fail_students = error_analysis_df[error_analysis_df['actual'] == 0]
pass_students = error_analysis_df[error_analysis_df['actual'] == 1]

# Insight 1: Đặc trưng quan trọng nhất
print(f"\n1. Các yếu tố quan trọng nhất ảnh hưởng đến kết quả học tập:")
print(f"   - Top 5 đặc trưng quan trọng: {', '.join(top_features)}")
if 'G2' in top_features or 'G1' in top_features:
    print(f"   - Điểm số kỳ trước (G1, G2) là yếu tố dự báo mạnh nhất")
    print(f"   - Khuyến nghị: Theo dõi sát sao điểm số từ đầu năm học để can thiệp sớm")

# Insight 2: Phân tích lỗi dự đoán
print(f"\n2. Phân tích nhóm sinh viên có nguy cơ cao:")
false_negatives = error_analysis_df[(error_analysis_df['actual'] == 1) & (error_analysis_df['predicted'] == 0)]
if len(false_negatives) > 0:
    avg_failures = false_negatives['failures'].mean()
    avg_absences = false_negatives['absences'].mean()
    print(f"   - Có {len(false_negatives)} sinh viên bị dự đoán sai (thực tế Pass nhưng dự đoán Fail)")
    print(f"   - Trung bình số lần thất bại: {avg_failures:.2f}, số ngày nghỉ: {avg_absences:.2f}")
    print(f"   - Khuyến nghị: Cần xem xét thêm các yếu tố định tính khác ngoài dữ liệu số")

# Insight 3: Hiệu suất mô hình
best_f1 = comparison_df['f1'].max()
best_accuracy = comparison_df['accuracy'].max()
print(f"\n3. Hiệu suất dự đoán của mô hình:")
print(f"   - Độ chính xác tốt nhất: {best_accuracy:.1%}")
print(f"   - F1-score tốt nhất: {best_f1:.3f}")
if best_f1 >= 0.85:
    print(f"   - Mô hình có độ tin cậy cao, có thể triển khai thực tế")
    print(f"   - Khuyến nghị: Xây dựng hệ thống cảnh báo sớm tự động")
else:
    print(f"   - Mô hình cần cải thiện thêm trước khi triển khai")
    print(f"   - Khuyến nghị: Thu thập thêm dữ liệu và đặc trưng mới")

# Insight 4: So sánh supervised vs semi-supervised
print(f"\n4. Hiệu quả của học bán giám sát:")
if semisupervised_f1 > supervised_f1:
    improvement = ((semisupervised_f1 - supervised_f1) / supervised_f1) * 100
    print(f"   - Học bán giám sát cải thiện {improvement:.1f}% so với chỉ dùng 5% nhãn")
    print(f"   - Có thể tận dụng dữ liệu không nhãn để cải thiện mô hình")
    print(f"   - Khuyến nghị: Áp dụng khi chi phí gán nhãn cao")
else:
    print(f"   - Với dữ liệu này, supervised learning đã đủ hiệu quả")
    print(f"   - Khuyến nghị: Tập trung vào chất lượng dữ liệu có nhãn")

# Insight 5: Tỷ lệ lỗi và ứng dụng thực tế
print(f"\n5. Ứng dụng thực tế và hạn chế:")
print(f"   - Tỷ lệ dương tính giả (FPR): {false_positive_rate:.1%}")
print(f"   - Tỷ lệ âm tính giả (FNR): {false_negative_rate:.1%}")
if false_negative_rate < false_positive_rate:
    print(f"   - Mô hình có xu hướng dự đoán Pass nhiều hơn (ít cảnh báo sai)")
    print(f"   - Khuyến nghị: Phù hợp cho hệ thống cảnh báo sớm, ít gây lo lắng thừa")
else:
    print(f"   - Mô hình có xu hướng dự đoán Fail nhiều hơn (nhiều cảnh báo)")
    print(f"   - Khuyến nghị: Cần kết hợp với đánh giá của giáo viên để tránh gây áp lực")

## 10. Phân Tích Mô Hình Và So Sánh

In [ ]:
print("=== PHÂN TÍCH SO SÁNH CÁC MÔ HÌNH ===")

# So sánh chi tiết giữa các mô hình
print(f"\nSo sánh mô hình:")
for name, metrics in results.items():
    print(f"\n{name}:")
    print(f"  - Accuracy: {metrics['accuracy']:.3f}")
    print(f"  - F1-score: {metrics['f1']:.3f}")
    print(f"  - ROC-AUC: {metrics['roc_auc']:.3f}")
    
    # Nhận xét
    if metrics['f1'] == comparison_df['f1'].max():
        print(f"  - Là mô hình tốt nhất theo F1-score")
    if metrics['roc_auc'] == comparison_df['roc_auc'].max():
        print(f"  - Có ROC-AUC tốt nhất")
    if name == 'Random Forest':
        print(f"  - Có khả năng xử lý đặc trưng phi tuyến tốt")
    elif name == 'Logistic Regression':
        print(f"  - Có tính giải thích cao, dễ hiểu")
    elif name == 'SVM':
        print(f"  - Hiệu quả với dữ liệu có ranh giới phức tạp")

# Đánh giá mô hình tốt nhất
best_metrics = results[best_model_name]
print(f"\n\nMÔ HÌNH TỐT NHẤT: {best_model_name}")
print(f"- F1-score: {best_metrics['f1']:.3f}")
print(f"- ROC-AUC: {best_metrics['roc_auc']:.3f}")
print(f"- Ứng dụng thực tế: Có thể sử dụng để dự đoán sớm nguy cơ trượt môn")
print(f"- Độ tin cậy: {best_metrics['accuracy']:.1%} trên tập kiểm thử")

In [ ]:
print("=== ĐÁNH GIÁ TỔNG THỂ THEO RUBRIC ===")
